In [1]:
#Peak fitting for CdSe (UV-Vis)
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import pybaselines as pybase
import csv

In [2]:
#Import Wavelengths as np array
with open(r"UV-visWavelengths.csv") as UV_WL:
    wavelengths = np.loadtxt(UV_WL, delimiter=",")
    electronvolts = 1239.84193/wavelengths

In [3]:
#Import Intensity data
from numpy import genfromtxt
data = genfromtxt(r"CdSe UV DoE.csv", delimiter=',')
Abs_data = np.nan_to_num(data, nan=0, posinf=0)

In [5]:
### Find lowest energy peak in spectra
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as mpl
import pybaselines as pybase

def findUV(y,wavelengths):
    #clean data to remove nan and inf
    m = np.stack((wavelengths,y),axis=0)
    m = np.transpose(m)
    m = (m[~np.isnan(m).any(axis=1), :])
    m = (m[~np.isinf(m).any(axis=1), :])
    wavelengths = m[:,0]
    y = m[:,1]
    #Smooth curves with Whittaker Eilers
    [y_base,_] = pybase.whittaker.aspls(y, lam=1000000000.0, diff_order=2, max_iter=100, tol=0.001, weights=None, alpha=None)
    [y_filt,_] = pybase.whittaker.aspls(y, lam=100000.0, diff_order=2, max_iter=100, tol=0.001, weights=None, alpha=None)
    #Subtract the minimum (<600nm)
    y = y - np.ndarray.min(y_base[550:1450])
    y_filt = y_filt - np.ndarray.min(y_base[550:1450])
    #Take derivative and smooth with Whittaker Eilers
    dydx = np.diff(y_filt)/np.diff(wavelengths)
    [dydx_filt,_] = pybase.whittaker.aspls(dydx, lam=100000.0, diff_order=2, max_iter=100, tol=0.001, weights=None, alpha=None)
    #Take derivative and smooth with Whittaker Eilers (double derivative)
    d2ydx2 = np.diff(dydx_filt)/np.diff(wavelengths)[1:]
    [d2ydx2_filt,_] = pybase.whittaker.aspls(d2ydx2, lam=50000.0, diff_order=2, max_iter=100, tol=0.001, weights=None, alpha=None)
    d2ydx2_raw = np.diff(dydx)/np.diff(wavelengths)[1:]
    d2ydx2_raw_cut = d2ydx2_raw[550:1450]
    d2ydx2_cut = d2ydx2_filt[550:1450]
    #Find peaks in wavelength range
    #Find peaks from raw curve (if peaks exist)
    from scipy.signal import find_peaks, peak_prominences
    peaks, _ = find_peaks(y_filt[550:1450])
    prominences = peak_prominences(y_filt[550:1450], peaks)[0]
    abs_heights = y_filt[550:1450][peaks]
    #Sort by most prominent peaks (prominences, wavelengths, y_filt heights, peak indices, zero column, peaks (in y_filt))
    a1 = np.stack((prominences,wavelengths[550:1450][peaks],abs_heights,peaks,np.zeros(np.shape(peaks)[0]),np.zeros(np.shape(peaks)[0]),y_filt[550:1450][peaks]),axis=0)
    a1 = np.transpose(a1)
    a1 = np.delete(a1, np.where(a1[:,-1] <= 0.001)[0], axis=0)
    a1 = a1[a1[:, 0].argsort()]

    peaks, _ = find_peaks(-d2ydx2_cut)
    prominences = peak_prominences(-d2ydx2_cut, peaks)[0]
    abs_heights = -d2ydx2_cut[peaks] 
    #Sort by most prominent peaks (prominences, wavelengths, d2ydx2 heights, peak indices, zero column, peaks (in y_filt))
    a2 = np.stack((prominences,wavelengths[550:1450][peaks],abs_heights,peaks,np.zeros(np.shape(peaks)[0]),np.zeros(np.shape(peaks)[0]),y_filt[550:1450][peaks]),axis=0)
    a2 = np.transpose(a2)
    a2 = np.delete(a2, np.where(a2[:,-1] <= 0.001)[0], axis=0)
    a2 = a2[a2[:, 0].argsort()]

    
    #Remove peaks right of most prominent peak
    sorted_peaks1 = a1[a1[:,1] <= a1 [-1:,1]]
    sorted_peaks2 = a2[a2[:,1] <= a2 [-1:,1]]
    if np.shape(a1)[0] > 0:
        sorted_peaks = np.stack((sorted_peaks1[-1,:]  , sorted_peaks2[-1,:]))
    if np.shape(a1)[0] == 0:
        sorted_peaks = sorted_peaks2
    #Get the two other peaks left of most prominent peak
    b = np.zeros(7)
    if np.shape(sorted_peaks)[0] > 0:
        b = sorted_peaks[sorted_peaks[:, 3].argsort()]
        b = b[-1,:]  
    #print(b)
    #Estimate fwhm using points where curve cuts zero
    if np.shape(sorted_peaks)[0] > 0:
        zero_crossings = np.where(np.diff(np.sign(d2ydx2_cut)))[0]
        idx = zero_crossings[np.argmax(zero_crossings>b[3])]
        fwhm = 2*(wavelengths[550:1450][idx]-b[1])
        b[4] = idx
        b[5] = fwhm   
    #Extract parameters of wavelength (mu), fwhm (nm) and height
    #Sort Parameters
    est_parameters = [b[1],b[5],b[6]]
    #print(est_parameters)

    #Plot smoothed Abs
    fig = plt.figure(figsize=(4, 4), dpi=80)
    plt.title('Smoothed Abs')
    plt.plot(wavelengths[550:1450],y[550:1450])
    plt.plot(wavelengths[550:1450],y_filt[550:1450])
    
    # Function to model and create data
    def gaussian(x, mu, fwhm, height):
        sig = abs(fwhm/2.35482004503)
        return height*np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
    # Generating clean data
    k = [b[3], b[4]]
    k = np.int_(k)
    x = wavelengths[550:1450][k[0]-20:k[1]+120]
    est = gaussian(x, b[1], b[5], b[6])
    #yn = y_filt[550:1450][k[0]:k[1]+100]
    yn = y[550:1450][k[0]-20:k[1]+120]

    # Executing curve_fit on noisy data
    try:
        popt, pcov = curve_fit(gaussian, x, yn, [b[1], abs(b[5]), b[6]])
        return([popt[0],popt[1],popt[2],1239.84193/popt[0],2*((1239.84193/popt[0])-1239.84193/(popt[0]+0.5*popt[1])),popt[2]])
    except:
        return([0,0,0,0,0,0])
    return(fig)

In [ ]:
#Peak fit on dataset
from scipy.signal import find_peaks, peak_prominences
np.set_printoptions(edgeitems=8, precision=3, suppress=True, threshold=5)
parameters = np.zeros((np.shape(Abs_data)[0],13))
index = 0
for row in Abs_data:
    print(index+1)
#Baseline correction to compare with first UV spectra for every reaction
    if np.remainder(index+1, 20) == 1: 
        Abs0 = Abs_data[index,7:]
        [Abs0_filt,_] = pybase.whittaker.aspls(Abs0, lam=100000000.0, diff_order=2, max_iter=100, tol=0.001, weights=None, alpha=None)
        Abs2 = Abs_data[index+1,7:]
        [Abs2_filt,_] = pybase.whittaker.aspls(Abs2, lam=100000000.0, diff_order=2, max_iter=100, tol=0.001, weights=None, alpha=None)
        Abs_diff = Abs2_filt - Abs0_filt
        a = np.ndarray.min(Abs_diff[550:1450])
        Abs_diff = Abs_diff-a
        idx_max = np.argmax(Abs_diff[550:1450])
        print(idx_max)
        idx_cut = np.argmax(Abs_diff[550+idx_max:1450]<0.002)
        print(idx_cut)
        master_idx = idx_cut + 200
        Abs = Abs0
    if np.remainder(index+1, 20) != 1: 
        master_idx = idx_cut + 100
        Abs = Abs_data[index,7:]
    
    Baseline_correction = Abs0_filt
    Baseline_correction[0:550+master_idx] = Abs0_filt[550+master_idx]
    Abs_corrected = Abs - Baseline_correction

    
    Analyze_WL = findUV(Abs_corrected,wavelengths)
    parameters[index,:7]=Abs_data[index,:7]
    parameters[index,7:13] = Analyze_WL
    print(parameters[index,:7])
    print(parameters[index,7:13])
    #time.sleep(1)
    plt.show()
    index += 1

In [7]:
#Save raw data
np.savetxt("CdSe UV fitting (DoE raw).csv", parameters, delimiter=",")

In [8]:
#Import raw data
from numpy import genfromtxt
data = genfromtxt(r"CdSe UV fitting (DoE raw).csv", delimiter=',')
Fit_data = abs(np.nan_to_num(data, nan=0, posinf=0))

In [9]:
#Cleaning of data to remove points which are unreasonable
index = 0
for row in Fit_data:
    if Fit_data[index,7] == 0 or Fit_data[index,10] == 0: #Check for zeros
        Fit_data[index,:] = np.zeros((1,13))
    elif Fit_data[index,7] < 400 or Fit_data[index,7] > 630: #Check feasibility of wavelength
        Fit_data[index,:] = np.zeros((1,13))
    elif abs(Fit_data[index,8]) < 18 or abs(Fit_data[index,8]) > 80: #Check feasibility of FWHM
        Fit_data[index,:] = np.zeros((1,13))
    elif Fit_data[index,11] > 0.5: #Check feasibility of FWHM in eV
        Fit_data[index,:] = np.zeros((1,13))
    elif Fit_data[index,12] > 1.2 or Fit_data[index,12] < 0.02: #Check feasibility of height in eV
        Fit_data[index,:] = np.zeros((1,13))
    index += 1
Clean_fit = Fit_data[~np.all(Fit_data == 0, axis=1)]

In [10]:
#Save cleaned data
np.savetxt("CdSe UV fitting (DoE cleaned).csv", Clean_fit, delimiter=",",fmt='%1.3f')